# Ejercicio 6

Implementar un detector de color YELLOW que funciones en tiempo real en un video. Para ello, implementar el algoritmo en OpenCV, de modo que capture las im´agene directamente de la c´ amara de su PC, y que muestre el resultado de la detecci´on en tiempo real en pantalla. Si usted lo desea, puede dise˜ nar el detector para que detecte otros colores diferentes. Mostrar en su informe, resultados (screenshots) del funcionamiento de su detector.

## Librerias

In [1]:
import cv2 as cv
import numpy as np
from PIL import Image

## Funciones auxiliares

In [2]:
def get_limits(color, delta):
    """
    Obtener los límites de un color en el espacio HSV

    Args:
        color (list): color en BGR
        delta (int): margen de error

    Returns:
        np.array: color en HSV
        np.array: límite inferior
        np.array: límite superior
    """
    # Convertir el color a HSV
    rgb_color = np.uint8([[color]])
    hsv_color = cv.cvtColor(rgb_color, cv.COLOR_BGR2HSV)

    # Definir los limites del color en HSV
    h_value = int(hsv_color[0][0][0])
    min_limit = np.array([max(h_value - delta, 0), 100, 100], dtype=np.uint8)
    max_limit = np.array([min(h_value + delta, 179), 255, 255], dtype=np.uint8)

    return min_limit, max_limit

In [ ]:
def gen_bbox(hsv_image, color_name, bgr_color, delta):
    """
    Generar una mascara y un bounding box para un color en una imagen

    Args:
        hsv_image (np.array): imagen en HSV
        color_name (str): nombre del color
        bgr_color (list): color en BGR
        delta (int): margen de error

    Returns:
        np.array: imagen con el bounding box
    """
    # Obtener los limites del color en HSV
    lower_limit, upper_limit = get_limits(bgr_color, delta)
    # Generar la mascara del color
    mask = cv.inRange(hsv_image, lower_limit, upper_limit)
    result = Image.fromarray(mask)
    # Encontrar el bounding box del color
    bbox = result.getbbox()

    # Si la bounding box no es None
    if bbox is not None:
        # Obtener las coordenadas del bounding box
        x1, y1, x2, y2 = bbox
        # Dibujar el bounding box
        cv.rectangle(hsv_image, (x1, y1), (x2, y2), bgr_color, 5)
        # Escribir el nombre del color
        cv.putText(hsv_image, color_name, (x1, y1), cv.FONT_HERSHEY_SIMPLEX, 1, bgr_color, 2, cv.LINE_AA)

    return hsv_image

## Captura de video y deteccion de colores

In [ ]:
# Definir los colores en BGR
YELLOW = [0, 255, 255]
RED = [0, 0, 255]
GREEN = [0, 255, 0]
BLUE = [255, 0, 0]

In [ ]:
# Inicializar la captura de video
webcam = cv.VideoCapture(0)

# Iniciar el bucle para capturar frames
while True:
    # Leer un frame de la captura
    ret, frame = webcam.read()
    # Convertir el frame a HSV para analizar los colores
    hsv_frame = cv.cvtColor(frame, cv.COLOR_BGR2HSV)
    
    # Generar el bounding box para cada color
    hsv_frame = gen_bbox(hsv_frame, 'Yellow', YELLOW, 15)
    hsv_frame =gen_bbox(hsv_frame, 'Red', RED, 15)
    hsv_frame =gen_bbox(hsv_frame, 'Green', GREEN, 15)
    hsv_frame =gen_bbox(hsv_frame, 'Blue', BLUE, 15)

    # Mostrar el frame con los bounding box
    cv.imshow('Webcam', hsv_frame)

    # Salir del bucle si se presiona la tecla 'q'
    if cv.waitKey(1) & 0xFF == ord('q'):
        break

# Liberar la captura de video y cerrar las ventanas
webcam.release()
cv.destroyAllWindows()